## 1. Load NERs extracted previously

In [19]:
import json

with open("../out/prompting_ners_gpt_4.1_mini.json", "r", encoding="utf-8") as fr:
    prompting_ners = json.load(fr)
    
with open("../out/spacy_ners.json", "r", encoding="utf-8") as fr:
    spacy_ners = json.load(fr)

In [20]:
prompt_ner_cnt = 0
spacy_ner_cnt = 0

print("Prompting:")
for key in prompting_ners.keys():
    print(f"\t- {key}: {len(prompting_ners[key])}")
    prompt_ner_cnt += len(prompting_ners[key])

print(f"\nNumber of NERs with Prompting: {prompt_ner_cnt}\n")

print("Spacy:")
for key in spacy_ners.keys():
    print(f"\t- {key}: {len(spacy_ners[key])}")
    spacy_ner_cnt += len(spacy_ners[key])

print(f"\nNumber of NERs with SpaCy: {spacy_ner_cnt}")


Prompting:
	- PER: 68
	- ORG: 6
	- DATE: 34
	- GPE: 9
	- WORK_OF_ART: 40
	- LOC: 7
	- EVENT: 2

Number of NERs with Prompting: 166

Spacy:
	- PER: 107
	- MISC: 65
	- LOC: 36
	- ORG: 2

Number of NERs with SpaCy: 210


NERs extracted by prompting appears more precise than SpaCy.

In [21]:
for key, values in prompting_ners.items():
    for ner in values:
        found = False
        for spacy_key, prompt_values in spacy_ners.items():
            if ner in prompt_values:
                if key != spacy_key:
                    print(f"'{ner}' with different key: '{key}' in Prompting and '{spacy_key}' in SpaCy")
                break
        else:
            print(f"'{ner}' found in Prompting under key '{key}' but not in SpaCy")

'El Capitán Medrano' found in Prompting under key 'PER' but not in SpaCy
'I. Arellano' found in Prompting under key 'PER' but not in SpaCy
'M. Vitse' with different key: 'PER' in Prompting and 'MISC' in SpaCy
'Sebastián de Prado' found in Prompting under key 'PER' but not in SpaCy
'Christiane Faliu-Lacourt' found in Prompting under key 'PER' but not in SpaCy
'Miguel de Orozco' found in Prompting under key 'PER' but not in SpaCy
'V. García Ruiz' found in Prompting under key 'PER' but not in SpaCy
'Alonso de Olmedo' found in Prompting under key 'PER' but not in SpaCy
'Subirá' with different key: 'PER' in Prompting and 'LOC' in SpaCy
'Tamesis Books' with different key: 'ORG' in Prompting and 'MISC' in SpaCy
'Cofradía de Nuestra Señora de la Novena' with different key: 'ORG' in Prompting and 'LOC' in SpaCy
'Genealogía' with different key: 'ORG' in Prompting and 'MISC' in SpaCy
'Reichenberger' with different key: 'ORG' in Prompting and 'LOC' in SpaCy
'Biblioteca Nacional de Madrid' with dif

In [22]:
for key, values in spacy_ners.items():
    for ner in values:
        found = False
        for prompt_key, prompt_values in prompting_ners.items():
            if ner in prompt_values:
                if key != prompt_key:
                    print(f"'{ner}' with different key: '{key}' in SpaCy and '{prompt_key}' in Prompting")
                break
        else:
            print(f"'{ner}' found in SpaCy under key '{key}' but not in Prompting")

'Lope' found in SpaCy under key 'PER' but not in Prompting
'Doctor Juan Rana' found in SpaCy under key 'PER' but not in Prompting
'Rana' found in SpaCy under key 'PER' but not in Prompting
'Rey.- Cómo' found in SpaCy under key 'PER' but not in Prompting
'Capitán Medrano' found in SpaCy under key 'PER' but not in Prompting
'Cosme' found in SpaCy under key 'PER' but not in Prompting
'Mateo' found in SpaCy under key 'PER' but not in Prompting
'mui zelebrado' found in SpaCy under key 'PER' but not in Prompting
'Rey.- ¿Qué' found in SpaCy under key 'PER' but not in Prompting
'Solís' found in SpaCy under key 'PER' but not in Prompting
'Ana' found in SpaCy under key 'PER' but not in Prompting
'Séneca' found in SpaCy under key 'PER' but not in Prompting
'Antonio Escamilla' found in SpaCy under key 'PER' but not in Prompting
'Cid Campeador' found in SpaCy under key 'PER' but not in Prompting
'Moreto' found in SpaCy under key 'PER' but not in Prompting
'Miguel de] Orozco' found in SpaCy under ke

## 2. Preparing LLM for RE

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [ ]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    introduction = fr.read()

### 2.1. First approach: Analyze the entire text passing NERs and text into the prompt.

In [32]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "entities"],
    template="""
        Dado el siguiente texto y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Lista de entidades (NERs):
        {entities}

        Texto de entrada:
        {text}

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
    """,
)

In [33]:
response = llm.invoke(
    entire_text_prompt.invoke({
        "text": introduction,
        "entities": "\n".join(
            [
                f"{key}: {', '.join(values)}"
                for key, values in prompting_ners.items()
            ]
        ),
    })
).content

response

'(Cosme Pérez)-[es conocido como]-(Juan Rana)  \n(Cosme Pérez)-[es hijo de]-(Damián Pérez)  \n(Cosme Pérez)-[es hijo de]-(Isabel de Basto)  \n(Cosme Pérez)-[estuvo casado con]-(Bernarda Ramírez)  \n(Cosme Pérez)-[estuvo casado con]-(Bernarda Manuela)  \n(Cosme Pérez)-[estuvo casado con]-(María de Acosta)  \n(Cosme Pérez)-[tuvo hija]-(Francisca María Pérez)  \n(Cosme Pérez)-[fue recibido en]-(Cofradía de Nuestra Señora de la Novena)  \n(Cofradía de Nuestra Señora de la Novena)-[fue recibida por]-(Tomás Fernández)  \n(Cosme Pérez)-[fue popular en]-(su tiempo)  \n(Domingo Canejil)-[recibió el nombre de]-(Ranilla)  \n(Cosme Pérez)-[intervino ante]-(Rey)  \n(Cosme Pérez)-[tuvo parentesco con]-(Bárbara Coronel)  \n(Cosme Pérez)-[fue apreciado por]-(Rey)  \n(Cosme Pérez)-[fue apreciado por]-(familia real)  \n(Cosme Pérez)-[actuó en]-(La portería de las damas)  \n(María Teresa de Austria)-[se refería a]-(Juan Rana)  \n(Cosme Pérez)-[fue mencionado en]-(entremés de Avellaneda)  \n(Cosme Pérez)-

In [34]:
for relation in response.split("\n"):
    print(relation.strip())

(Cosme Pérez)-[es conocido como]-(Juan Rana)
(Cosme Pérez)-[es hijo de]-(Damián Pérez)
(Cosme Pérez)-[es hijo de]-(Isabel de Basto)
(Cosme Pérez)-[estuvo casado con]-(Bernarda Ramírez)
(Cosme Pérez)-[estuvo casado con]-(Bernarda Manuela)
(Cosme Pérez)-[estuvo casado con]-(María de Acosta)
(Cosme Pérez)-[tuvo hija]-(Francisca María Pérez)
(Cosme Pérez)-[fue recibido en]-(Cofradía de Nuestra Señora de la Novena)
(Cofradía de Nuestra Señora de la Novena)-[fue recibida por]-(Tomás Fernández)
(Cosme Pérez)-[fue popular en]-(su tiempo)
(Domingo Canejil)-[recibió el nombre de]-(Ranilla)
(Cosme Pérez)-[intervino ante]-(Rey)
(Cosme Pérez)-[tuvo parentesco con]-(Bárbara Coronel)
(Cosme Pérez)-[fue apreciado por]-(Rey)
(Cosme Pérez)-[fue apreciado por]-(familia real)
(Cosme Pérez)-[actuó en]-(La portería de las damas)
(María Teresa de Austria)-[se refería a]-(Juan Rana)
(Cosme Pérez)-[fue mencionado en]-(entremés de Avellaneda)
(Cosme Pérez)-[fue acusado de]-(pecado nefando)
(Cosme Pérez)-[fue an

In [35]:
print(f"Total de relaciones encontradas: {len(response.split('\n'))}")

Total de relaciones encontradas: 852
